In [2]:
#!pip install ..
!pip install emoji
import emoji

!pip list
# Importation des différentes librairies utiles pour le notebook

#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

!pip install seaborn
!pip install scikit-learn
!pip install optuna

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 4.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Package                   Version
------------------------- --------------
anyio                     4.9.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.4
bleach                    6.2.0
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
comm                      0.2.2
debugpy                   1.8.14
decorator                 5.2.1
defusedxml                0.7.1
emoji                     2.14.1
executing                 2.2.0
fastjsonschema            2.21.1
fqdn                      1.5.1
h11    

Matplotlib is building the font cache; this may take a moment.


In [19]:
df=pd.read_csv('scitweets_balanced.tsv', sep='\t')
display (df.head())

,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context
0,0.0,3.166700e+17,Knees are a bit sore. i guess that's a sign th...,0,0.0,0.0,0.0
1,1.0,3.190909e+17,McDonald's breakfast stop then the gym 🏀💪,0,0.0,0.0,0.0
2,2.0,3.220309e+17,Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0.0
3,3.0,3.226948e+17,Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0.0
4,4.0,3.285244e+17,Does daily routine help prevent problems with ...,1,1.0,0.0,0.0


In [20]:
!wget https://www.lirmm.fr/~poncelet/Ressources/StopWordsFrench.csv

zsh:1: command not found: wget


In [21]:
#code etienne matrice
import re
import numpy as np

stopwords_df = pd.read_csv("StopWordsFrench.csv", sep=',', index_col=0)
# Conversion en liste. Selon la structure, ici on récupère les index comme stopwords.
french_stopwords = stopwords_df.index.tolist()

vectorizer = TfidfVectorizer(
    min_df = 5,
    max_df = 0.95,
    stop_words=french_stopwords,
    lowercase=False,
    ngram_range=(1,2),
    max_features=50000
)
X_tfidf = vectorizer.fit_transform(df['text'])

tfidf_df = pd.DataFrame(
    X_tfidf.toarray(),
    columns=vectorizer.get_feature_names_out()
)

def is_valid_token(token):
    # Garder uniquement les mots en alphabet latin (minuscules/majuscules)
    # On rejette si c’est purement numérique ou contient des caractères spéciaux non latins
    if token.isdigit():
        return False
    if re.match(r'^[a-zA-Z]+$', token):  # Mot composé uniquement de lettres latines
        return True
    return False

# Filtrer les colonnes du DataFrame TF-IDF
filtered_columns = [token for token in tfidf_df.columns if is_valid_token(token)]
tfidf_filtered_df = tfidf_df[filtered_columns]

# Identifie les colonnes numériques (constituées uniquement de chiffres)
numeric_columns = [token for token in tfidf_df.columns if token.isdigit()]

# Crée une colonne 'numeric' qui est la somme des colonnes numériques
tfidf_df['numeric'] = tfidf_df[numeric_columns].sum(axis=1)

# Supprime les colonnes numériques individuelles
tfidf_df.drop(columns=numeric_columns, inplace=True)

# Supprimer aussi les colonnes avec des caractères non latins, comme avant
filtered_columns = [token for token in tfidf_df.columns if is_valid_token(token)]
tfidf_filtered_df = tfidf_df[filtered_columns + ['numeric']]

print("Taille de la matrice TF-IDF :", tfidf_df.shape)
display(tfidf_df)

Taille de la matrice TF-IDF : (1530, 1233)


,2VgsIg8Trp,AND,About,According,According to,Adversity,Adversity causes,Ain,All,America,...,years,you,you can,you know,you re,your,your heart,your mental,yours,numeric
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.218843,0.0,0.0,0.0,0.632992
